In [28]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.datasets import fetch_lfw_people
from sklearn.datasets import fetch_olivetti_faces
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.decomposition import PCA
from sklearn.svm import SVC
import matplotlib.pyplot as plt
import random

In [29]:
# ---------------------------- LOAD OLIVIETTA DATASET ---------------------------- #

def load_dataset():
    dataset = fetch_olivetti_faces(data_home='D:\Senior II\Image Processing\Project\Datasets/Olivietta Dataset', shuffle=True, random_state=47)
    return dataset

In [30]:
def extract_info_from_dataset(dataset, print=False):
    
    '''
    Returns: 
        - images: ndarray(m, height, width): the images of the dataset
        - m: int: the number of images in the dataset
        - height: int: the height of each image
        - width: int: the width of each image
        - total_images: int:  the total number of images in the dataset
        - n_features: int: the number of features of each image = height * width
        - y: ndarray(m,): the labels of the images
        - num_people: int the number of people in the dataset
    '''
    
    images = dataset.images

    m = images.shape[0]
    if(print):
        print('Images shape:',images.shape)

    total_images, height, width = images.shape
    if(print):
        print(f'Each image has size: {height} x {width}')
        print(70* '-')

    n_features = height*width
    if(print):
        print(f'N^2 = n_features = h x w = {n_features}')
        print(70* '-')

    y = dataset.target
    if(print):
        print('y has shape:', y.shape)

    num_people = np.max(y) + 1
    if(print):
        print('Number of people =', num_people)

    return images, m, height, width, total_images, n_features, y, num_people

In [31]:
#----------------------- CREATE PEOPLE DICTIONARY -----------------------#


def create_people_dict(total_images, num_people, y):
    # Key: person ID
    # Value: List of all person images indices
    person_image_dict = dict()

    for image_index in range(total_images):
        if (y[image_index] not in person_image_dict.keys()):
            person_image_dict[y[image_index]] = [image_index]
        else:
            person_image_dict[y[image_index]].append(image_index)
    return person_image_dict

In [32]:
# ------------------ VIEW RANDOM IMAGE ------------------ #

def view_rand_image(images, m):
    index = int(random.random() * m)
    index = 0
    image = images[index,:,:]
    # name = target_names[y[index]]

    # print(name)
    plt.imshow(image, cmap='gray')
    plt.show()

In [33]:
# --------------------- GET FLATTEN IMAGES --------------------- #

def flattened_images(images, m, height, width, print=False):
    # Flatten images array
    # Each column is an image

    # N^2 * M
    flattened_images = images.reshape(m, -1).T

    if(print):
        print(f'Shape of training images after flattening: {flattened_images.shape}')
    
    return flattened_images


In [34]:
# ----------------- CALCULATE AVERAGE IMAGE ----------------- #

def average_image(flattened_images, height, width, print=False):
    # In all corresponding pixels in all images, we calculate the average

    # N^2 * 1
    average_image = np.mean(flattened_images, axis = 1)[:,np.newaxis]

    if(print):
        print(f'Average image has shape: {average_image.shape}')
        plt.imshow(average_image.reshape(height,width), cmap='gray', title='Average Image')
        plt.show()
    return average_image

In [35]:
#-------------------------------- GET DIFFERENCE IMAGES --------------------------------#

def difference_images(flattened_images, average_image, print=False):
    # Subtract the average image from all images
    # This is done to remove the average face from all images
    # N^2 * M
    difference_images = flattened_images - average_image
    if(print):
        print(f'Shape after subtracting average face: {difference_images.shape}')


In [36]:
# --------------------- SHOW ONE DIFFERENCE IMAGE AND COMPARE TO ORIGINAL --------------------- #

def show_difference_image(difference_images, train_images, height, width, m):
    # Show one difference image and compare to original
    index = int(random.random() * m)

    plt.imshow(difference_images[:,index].reshape(height, width), cmap='gray')
    plt.show()

    # Print original image
    plt.imshow(train_images[index,:,:], cmap='gray')
    plt.show()

In [37]:
#--------------------- GET COVARIANCE MATRIX --------------------- #

def covariance_matrix(m, n_features, difference_images, print=False):
    # C = A * A^T where A = difference_images

    # N^2 * M
    A = difference_images

    if (m > n_features):
        # N^2 * N^2
        covariance_matrix = np.matmul(difference_images, difference_images.T)
        if(print):
            print('Shape of covariance matrix = N^2 * N^2 = ', covariance_matrix.shape)
        
    else:
        # M * M
        covariance_matrix = np.matmul(difference_images.T, difference_images)
        if(print):
            print('Shape of covariance matrix = M * M = ', covariance_matrix.shape)

In [38]:
# --------------------------------- GET EIGENVALUES AND EIGENVECTORS --------------------------------- #

def eigenvalues_eigenfaces(covariance_matrix, difference_images, print=False):
    # M eigenvalues and M eigenvectors
    # where M is the number of examples

    eigenvalues, eigenvectors = np.linalg.eigh(covariance_matrix)
    
    if(print):
        print('Shape of eigenvalues:', eigenvalues.shape)
        print('Shape of eigenvectors before matrix multiplication:', eigenvectors.shape)

    # Try to remove
    eigenfaces = np.matmul(difference_images, eigenvectors)

    if(print):
        print('Shape of eigenfaces after matrix multiplication:', eigenfaces.shape)

    return eigenvalues, eigenfaces

In [39]:
# --------------------------------- NORMALIZE eigenfaces --------------------------------- #

def normalize_eigenfaces(eigenfaces, print=False):
    eigenfaces = eigenfaces / np.linalg.norm(eigenfaces, axis=0)
    if(print):
        print('Shape of eigenfaces after normalization:', eigenfaces.shape)
    return eigenfaces

In [40]:
# ------------------------ SORT eigenfaces ------------------------ #

def sort_eigenvalues_eigenfaces(eigenvalues, eigenfaces, print=False):
    # Get top K eigenfaces
    indices_of_top_eigenvalues = np.argsort(-eigenvalues)
    eigenvalues = eigenvalues[indices_of_top_eigenvalues]
    eigenfaces = eigenfaces[:, indices_of_top_eigenvalues]
    return eigenvalues, eigenfaces

In [41]:
# --------------------------------- CALCULATE K --------------------------------- #

def calculate_K(eigenvalues, m, variance = 0.8, print = False):
    #Calculate the number of components to preserve specified variance
    K = m
    for ii, eigen_value_cumsum in enumerate(np.cumsum(eigenvalues) / np.sum(eigenvalues)):
        if eigen_value_cumsum > variance:
            K = ii
            break

    if(print):
        print(f'Number of components to preserve {variance*100}% of the variance = {K}')

In [42]:
# --------------------------------- SELECT K TOP eigenfaces --------------------------------- #


def select_K_top_eigenfaces(eigenvalues, eigenfaces, K, print=False):
    # Select only K eigenfaces
    eigenvalues = eigenvalues[:K].copy()
    eigenfaces = eigenfaces[:, :K].copy()

    if(print):
        print('Shape of eigenvalues after selecting top K:', eigenvalues.shape)
        # N^2 * K
        print('Shape of eigenfaces after selecting top K:', eigenfaces.shape)
    return eigenvalues,eigenfaces

In [43]:

# ------------------------ SHOW EIGENFACES ------------------------ #

def show_eigenfaces(eigenfaces, height, width, K, num_show=16):
    # Show eigenfaces
    for i in range(min(K, num_show)):
        plt.subplot(4, 4, i+1)
        plt.imshow(eigenfaces[:,i].reshape(height, width), cmap='gray')
        plt.title(f'Eigenface {i+1}')
        plt.axis('off')

In [44]:
# ------------------------ SHOW A RANDOM EIGENFACE ------------------------ #

def show_random_eigenface(eigenfaces, height, width, m):
    index = int(random.random() * m)
    plt.imshow(eigenfaces[:,index].reshape(height, width), cmap='gray')
    plt.show()

In [45]:
# ------------------------ CALCULATE OMEGAS ------------------------ #

def omegas(difference_images, eigenfaces, print=False):
    omegas = []
    for image in difference_images.T:
        omegas.append(np.dot(image, eigenfaces))
    omegas = np.array(omegas)
    if(print):
        print('Shape of omegas:', omegas.shape)
    return omegas

## Testing

In [46]:
def predict(omegas, test_image, average_image, eigenfaces, height, width, print=False):
    # Predict the class of a test image
    # Calculate omega for test image
    omega = np.dot(test_image - average_image, eigenfaces)

    # Calculate distance between omega and all omegas
    distances = np.linalg.norm(omegas - omega, axis=1)

    # Get the index of the minimum distance
    index = np.argmin(distances)

    return index

In [47]:
def recognizer_main(input_image):
    dataset = load_dataset()
    images, m, height, width, total_images, n_features, y, num_people = extract_info_from_dataset(dataset)
    #people_dict = create_people_dict(total_images, num_people, y)
    flattened_images = flattened_images(images, m, height, width)
    average_image = average_image(flattened_images, m, height, width)
    difference_images = difference_images(flattened_images, average_image)
    covariance_matrix = covariance_matrix(m, n_features, difference_images)
    eigenvalues, eigenfaces = eigenvalues_eigenfaces(covariance_matrix, difference_images)
    eigenfaces = normalize_eigenfaces(eigenfaces)
    eigenvalues, eigenfaces = sort_eigenvalues_eigenfaces(eigenvalues, eigenfaces)
    K = calculate_K(eigenvalues, m)
    eigenvalues, eigenfaces = select_K_top_eigenfaces(eigenvalues, eigenfaces, K)
    omegas = omegas(difference_images, eigenfaces)
    predicted_index = predict(omegas, input_image, average_image, eigenfaces, height, width)
    print('Predicted index:', predicted_index)
